In [ ]:
# Original file name: tis.ipynb from Rare Event Summer School in Dropbox
# Written by: Steven Hall 
# Modified by Porhouy (PH) Minh
# Last Updated Date: 03-18-2023 
# SAMPEL Group

## TIS Tutorial


- This code will perform transition interface sampling (TIS) on a simple potential energy surface (PES). 

- The code is constructed as follows: 

    - Choosing your PES

    - Deciding on your simulation settings

    - Run equilibration and basin simulation 

    - Generate an initial path at a higher temperature

    - Perform TIS on your selected PES at your desired temperature
        
    - Visualize the paths for each interface ensemble

    - Calculate rate for the event (A --> B)

        - Calculate flux through the first interface

        - Calculate and visualize the crossing probability (for each interface and cumulative)

## Learning Objectives

1. Understand what a transition interface sampling (TIS) is.  

    - How is TIS different from TPS?  
    </br>

2. Explore the importance of interface placement and interface type on a given potential energy surface (PES).
    * To do this exploration, you can modify the following variables in the code:

        - **pes_type**: this will change the PES that you use for your simulation 

        - **op_type**: this will change the form of your op. The options provided are op_type = x, y, or x+y

        - **interfaces values** in the interface list: this will change where you place your interface

        - **basinA; basinB**: this will change where you define your basin(s) on the PES

        - **init_coords**: this will change where you start on your PES in your simulation

        - **beta**: this will change the temperature since beta = 1/(k_B*T), which consequently also changes the PES barrier height due to the change in temperature (T)

    * Questions to think about while exploring: 

        - How does changing variables related to the PES/my system (like pes_type, beta, basinA, basinB) impact my sampling?
        
        - How does changing variables related to TIS methodology (like op_type, interfaces values, init_coords) impact my sampling?

3. Understand the procedure in TIS sampling and the reason behind these steps that we follow for performing a RETIS "simulation".

    - Why do we need to run an intial simulation at a higher temperature?


## Energy surface contours options

PES-1: $$ V(x,y) = 0.02(x^4+y^4) - 4\exp(-(x+2)^2 - (y+2)^2) - 4\exp(-(x-2)^2 - (y-2)^2) + 0.3(x-y)^2 + 0.0026 $$

<img src="https://raw.githubusercontent.com/minhx010/pathsampling-notebooks/main/PES-figures/pes-1.png"> 

PES-2: $$ V(x,y) = 0.03(x^4+y^4) - 4\exp(-(x+2)^2 - (y+2)^2) - 4\exp(-(x-2)^2 - (y-2)^2) + 0.4(x-y)^2 + 4\exp(-(x^2+y^2)) - 2.1245 $$

<img src="https://raw.githubusercontent.com/minhx010/pathsampling-notebooks/main/PES-figures/pes-2.png">

PES-3: $$ V(x,y) = 0.02(x^4+y^4) - 3.73\exp(-\frac{(x+2)^2}{8} - \frac{(y+2)^2}{8}) - 3.73\exp(-\frac{(x-2)^2}{8} - \frac{(y-2)^2}{8}) + 3\exp(-\frac{x^2}{2} - \frac{y^2}{15}) + 2\exp(-\frac{x^2}{2} - \frac{y^2}{2}) - 0.5085 $$

<img src="https://raw.githubusercontent.com/minhx010/pathsampling-notebooks/main/PES-figures/pes-3.png">

PES-4: Müller-Brown Potential 
$$ V_{MB}(x,y) = \sum_{i=1}^{4}{A_{i}\exp([ a_{i}(x-\bar{x_{i}})^2 + b_{i}(x-\bar{x_{i}})(y-\bar{y_{i}}) + c_{i}(y-\bar{y_{i}})^2 ])} $$ 
where:

$$ A = [-8, -4, -6.8, 0.6] $$
$$ a = [-0.111, -0.111, -0.722, 0.0778] $$
$$ b = [0, 0, 1.22, 0.0667] $$
$$ c = [-1.11, -1.11, -0.722, 0.0778] $$
$$ \bar{x} = [3, 0, -1.5, -3] $$
$$ \bar{y} = [-3, -1.5, 1.5, 0] $$

<img src="https://raw.githubusercontent.com/minhx010/pathsampling-notebooks/main/PES-figures/pes-4.png">

## Choose your PES

- Here you can plot the PES you want and confirm that is what you want. 

In [ ]:
import numpy as np
import sys
import math
import copy
from math import exp
import langevin_dynamics as ld
import random
import time
import matplotlib.pyplot as plt
from matplotlib import colors, ticker, cm

# Select your PES type: ***** EDIT HERE *****
pes_type = 1

# Plot potential energy surface contours
N = 100
x_vec = np.linspace(-3.5, 3.5, N)
y_vec = np.linspace(-3.5, 3.5, N)
X, Y = np.meshgrid(x_vec, y_vec)
energy = np.zeros((N, N))

# Plot contours
for i in range(len(x_vec)):
    for j in range(len(y_vec)):
        energy[j][i] = ld.potential(pes_type,x_vec[i],y_vec[j])
plt.contour(x_vec,y_vec,energy,np.linspace(-3,3,20), cmap = 'jet')
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'))
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
plt.xlabel('x')
plt.ylabel('y')
plt.show

## Simulation Settings

- Here you define what op_type you would like to use, where you will place your interfaces, where your basins are, your initial starting configuration (position and momentum) and the temperature of your simulation. 

In [ ]:
# TIS settings
n_moves = 1000              # number of moves to perform for each path ensemble - path ensemble corresponding to each diff interface
n_discard = 50              # number of moves to discard at the beginning of RETIS - we do this to decorrelate new paths from old paths
op_type = 1                 # order parameter;  1:x  2:y  3:x+y
interfaces = [-1.3, -0.8, -0.5, 0.0, 0.5, 1.0, 2.0]
basinA = -2.09              # boundary of basin A; must be less than the first interface
basinB = interfaces[-1]     # basin B defined as last interface
init_coords = [-1, -1]      # initial coordinates
init_p = [0, 0]             # initial momenta
basineqlen = 5000           # basin equilibration length (steps)
basinlen = 50000            # basin simulation length
sigma = 0.5                 # standard deviation of momenta perturbation

#Langevin dynamics settings
beta = 2.5                  # 1/kT
gamma = 5.0                 # friction
dt = 0.01                   # time step size



# Plot PES contours with interfaces
# Plot contours
for i in range(len(x_vec)):
    for j in range(len(y_vec)):
        energy[j][i] = ld.potential(pes_type,x_vec[i],y_vec[j])

plt.contour(x_vec,y_vec,energy,np.linspace(-3,3,20), cmap = 'jet')

# Plot basin boundaries

if op_type == 1:
    plt.plot(np.linspace(basinA,basinA,10),np.linspace(min(y_vec),max(y_vec),10),color='r', label='Basin A')
    plt.plot(np.linspace(basinB,basinB,10),np.linspace(min(y_vec),max(y_vec),10),color='b', label='Basin B')
elif op_type == 2:
    plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinA,basinA,10),color='r', label='Basin A')
    plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinB,basinB,10),color='b', label='Basin B')
else:
    xplot = np.linspace(min(x_vec),max(x_vec),10)
    yplmax = basinB - xplot
    yplmin = basinA - xplot
    plt.plot(xplot,yplmin,color='r', label='Basin A')
    plt.plot(xplot,yplmax,color='b', label='Basin B')

# Plot interfaces
for i in range(len(interfaces)-1):
    
    if op_type == 1:
        plt.plot(np.linspace(interfaces[i],interfaces[i],10),np.linspace(min(y_vec),max(y_vec),10), color='grey')
    elif op_type == 2:
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(interfaces[i],interfaces[i],10), color='grey')
    else:
        xplot = np.linspace(min(x_vec),max(x_vec),10)
        yplot = interfaces[i]-xplot
        plt.plot(xplot,yplot, color='grey')    
        
cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'))
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show

## Equilibration Run

- Here the initial point is being relaxed into basin A.

In [ ]:
# Declare array to store basin trajectory
basintraj = np.zeros((basinlen + 1, 6),dtype=float)                             # matrix: each row is the next configuration and each column is x, y, px, py, fx, fy

# Calculate initial forces
fx,fy = ld.force(init_coords[0],init_coords[1],init_p[0],init_p[1],dt,beta,gamma,pes_type)

# Combine positions, momenta, and forces to make an initial phase point
init_phasepoint = init_coords + init_p + [fx,fy]
basintrajeq = ld.vv_step(init_phasepoint,dt,beta,gamma,pes_type)                # use velocity verlet to generate the next configuration of x, y, px, py, fx, fy

# Equilibrate in basin
for i in range(1,basineqlen + 1):
    new_basintrajeq = ld.vv_step(basintrajeq,dt,beta,gamma,pes_type)            # array of x, y, px, py, fx, fy
    basintrajeq = new_basintrajeq
    op = ld.calc_op(op_type,basintrajeq[0],basintrajeq[1])                      #ld.calc_op(op_type, x, y of the trajectory)
    # check if trajectory reaches basin B
    if op >= basinB:
        sys.exit("Basin trajectory reached B! Exiting...")


In [ ]:
# Visualizing equilibration 

# for i in range(len(x_vec)):
#     for j in range(len(y_vec)):
#         energy[j][i] = ld.potential(pes_type,x_vec[i],y_vec[j])
 
# plt.contour(x_vec,y_vec,energy,np.linspace(-3,3,20), cmap = 'jet')

# plt.scatter(basintraj[:,0],basintraj[:,1]) 

# cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'))
# cbar.set_ticks([])
# cbar.set_label(label = 'Energy', size=12)
# plt.xlabel('x')
# plt.ylabel('y')
# plt.show


## Run a basin simulation

- Why do we need to run basin simulation?

**[Answer: Edit Text]**

In [ ]:
basintraj[0] = basintrajeq                                                      # saving the last configuration from the equilibration run as the first config for prod run
fromBasin = False
n_cross_flux = 0

# Run one basin A simulation and check for first crossings
for j in range(1,basinlen + 1):
    basintraj[j] = ld.vv_step(basintraj[j-1],dt,beta,gamma,pes_type)            # each iteration is a configuration so that basintraj at the end makes a matrix
    op = ld.calc_op(op_type,basintraj[j][0],basintraj[j][1]) 
    
    if op < basinA:
        fromBasin = True
    
    if fromBasin == True and op >= interfaces[0]: 
        n_cross_flux += 1
        fromBasin = False
    
    # Check if trajectory reaches basin B
    if op >= basinB:
        sys.exit("Basin trajectory reached B! Exiting...")

if n_cross_flux == 0:
    sys.exit("No first crossings obtained from basin A to interface 0. Exiting...")


# Visualizing basin simulation 

for i in range(len(x_vec)):
    for j in range(len(y_vec)):
        energy[j][i] = ld.potential(pes_type,x_vec[i],y_vec[j])

plt.contour(x_vec,y_vec,energy,np.linspace(-3,3,20), cmap = 'jet')

# Plot basin boundaries

if op_type == 1:
    plt.plot(np.linspace(basinA,basinA,10),np.linspace(min(y_vec),max(y_vec),10),color='r', label='Basin A')
    plt.plot(np.linspace(basinB,basinB,10),np.linspace(min(y_vec),max(y_vec),10),color='b', label='Basin B')
elif op_type == 2:
    plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinA,basinA,10),color='r', label='Basin A')
    plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinB,basinB,10),color='b', label='Basin B')
else:
    xplot = np.linspace(min(x_vec),max(x_vec),10)
    yplmax = basinB - xplot
    yplmin = basinA - xplot
    plt.plot(xplot,yplmin,color='r', label='Basin A')
    plt.plot(xplot,yplmax,color='b', label='Basin B')

plt.plot(basintraj[:,0],basintraj[:,1],color='darkviolet') 

cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'))
cbar.set_ticks([])
cbar.set_label(label = 'Energy', size=12)
plt.legend()
plt.xlabel('x')
plt.ylabel('y')
plt.show


## Generate the initial paths by simulating at a higher temperature

- Why do we need to generate an initial path at a higher temperature? **Bonus**: How do we control the temperature? 

**[Answer: Edit Text]**

In [ ]:
# Run at higher temperature to generate an initial path
initpath = []
beta_initpath = 1.5                                                     # this controls the temperature because beta = 1/kb*T
fromBasin = False

# Use init_phasepoint to start trajectory
trajstep = ld.vv_step(init_phasepoint,dt,beta_initpath,gamma,pes_type)  # array of x, y, px, py, fx, fy to get the first config to generate a path
op = ld.calc_op(op_type,trajstep[0],trajstep[1])                        # calculate the op to see where this config is at, in A or pass A and cross some lambda, or in B?

while op < basinB:                                                      # as long as it is not in B
    nextstep = ld.vv_step(trajstep,dt,beta_initpath,gamma,pes_type)              # generate next config
    trajstep = nextstep                                                 # save this config to trajstep, so we could shoot from it again 
    op = ld.calc_op(op_type,trajstep[0],trajstep[1])

    if op < basinA:                                                     # if this then this traj is from basin A 
        fromBasin = True
        initpath = []                                                   # reset path if it returns to A - do this until it's about to leave the basin
        trajstep = np.append(trajstep,[op],axis=0)                      # array of x, y, px, py, fx, fy and op value
        initpath.append(trajstep) 
        continue

    if op >= basinA and fromBasin == True:                              # now that you have left basin A, we'll save the different configs to the list initpath
        trajstep = np.append(trajstep,[op],axis=0)
        initpath.append(trajstep)
        continue

initpath = np.asarray(initpath)                                         # this gives all configurations of this 1 initial path
allpaths = [[] for i in range(len(interfaces)-1)]                       # list of empty lists of size len(interfaces)-1, each empty list is a placeholder for paths in 1 interface 
acc = np.zeros(len(interfaces)-1)
n_cross = np.zeros(len(interfaces)-1)

for i in range(len(interfaces)-1):                                      # loop over amt of interface starting from 0 
    allpaths[i].append(initpath)    


# Visualize initial paths for each interface

for i in range(len(interfaces) - 1):
    plt.figure()
    
    plt.contour(x_vec,y_vec,energy,np.linspace(-3,3,20), cmap = 'jet')
    
    for j in range(0,len(allpaths[i])):
        plt.plot(allpaths[i][j][:,0],allpaths[i][j][:,1],color='darkviolet')               # allpaths[i] tells you the interface ensemble you're in, 
                                                                                           # [j] tells you a specific path in that ensemble, [:,0/1] 
                                                                                           # takes all the x/y values of the paths in i   
    # Plot basin boundaries

    if op_type == 1:
        plt.plot(np.linspace(basinA,basinA,10),np.linspace(min(y_vec),max(y_vec),10),color='r', label='Basin A')
        plt.plot(np.linspace(basinB,basinB,10),np.linspace(min(y_vec),max(y_vec),10),color='b', label='Basin B')
    elif op_type == 2:
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinA,basinA,10),color='r', label='Basin A')
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinB,basinB,10),color='b', label='Basin B')
    else:
        xplot = np.linspace(min(x_vec),max(x_vec),10)
        yplmax = basinB - xplot
        yplmin = basinA - xplot
        plt.plot(xplot,yplmin,color='r', label='Basin A')
        plt.plot(xplot,yplmax,color='b', label='Basin B')

    # Plot interfaces
    if op_type == 1:
        plt.plot(np.linspace(interfaces[i],interfaces[i],10),np.linspace(min(y_vec),max(y_vec),10), color='grey')
    elif op_type == 2:
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(interfaces[i],interfaces[i],10), color='grey')
    else:
        xplot = np.linspace(min(x_vec),max(x_vec),10)
        yplot = interfaces[i]-xplot
        plt.plot(xplot,yplot, color='grey')

    cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'))
    cbar.set_ticks([])
    cbar.set_label(label = 'Energy', size=12)
    plt.legend()
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show 

## Perform TIS

- In this section, to perform TIS, the program will perform a shooting move for each interface. 

    - In the shooting move, you first select a random configuration along the previous accepted path. Then you will randomize the configuration's momenta and afterwards, integrate the equation of motion forward and backward in time until one end reaches A and the other end reaches A or B. This will either form an AA path or an AB path. 

    <img src="https://raw.githubusercontent.com/SAMPEL-Group/pathsampling-tutorials/main/shoot-moves/2-ways-shoot.png">


In [ ]:
starttime = time.monotonic()

for move in range(1, n_moves):
    print('Move {}'.format(move), end='\r')
    
    for i in range(len(interfaces)-1):
        path = allpaths[i][move-1]
        reject_move = False
        lmax = round((len(path) + 1)/np.random.uniform())
        
        # Pick random shooting point
        index = np.random.randint(len(path) - 2) + 1                        # minus 2 to avoid basinA and i+1 points
        
        # Perturb the momenta
        shoot_point = copy.deepcopy(path[index])
        shoot_point[2] += np.random.normal(0,sigma)
        shoot_point[3] += np.random.normal(0,sigma)
        trial_path = np.asarray([shoot_point])
        if exp(-beta*(ld.kinetic_energy(shoot_point) - ld.kinetic_energy(path[index]))) < np.random.uniform():
            reject_move = True
        
        # Integrate backwards path if we have not rejected the move...
        if reject_move == False:

            path_length = 1
            trial_path[:,2] *=-1
            trial_path[:,3] *=-1
            trajstep = copy.deepcopy(trial_path[-1])
            op = ld.calc_op(op_type,trajstep[0],trajstep[1])
            
            while op >= basinA:

                trajstep = ld.vv_step(trajstep[:6],dt,beta,gamma,pes_type)
                op = ld.calc_op(op_type,trajstep[0],trajstep[1])
                trajstep = np.append(trajstep,[op],axis=0)
                trial_path = np.append(trial_path,[trajstep],axis=0)
                path_length +=1
                
                # Reject if the maximum path length is exceeded
                if path_length > lmax:
                    reject_move = True
                    break

                # Reject if the backward path segment goes to B
                if op >= basinB:
                    reject_move = True
                    break
        
        # Forward shooting             
        if reject_move == False: 
            trial_path = np.flip(trial_path,axis=0)                             # flip it from the back to front of this trial path (because it was shooting backwards), 
                                                                                # so we flip so that the last config we extract from here will be where we'll use to shoot forward
            trial_path[:,2] *=-1
            trial_path[:,3] *=-1
            trajstep = copy.deepcopy(trial_path[-1])
            op = ld.calc_op(op_type,trajstep[0],trajstep[1])
    
            while op >= basinA and op < basinB:                                 # this is so that it restricts forward shooting to either end in A or B

                nextstep = ld.vv_step(trajstep[:6],dt,beta,gamma,pes_type)
                trajstep = nextstep
                op = ld.calc_op(op_type,trajstep[0],trajstep[1])
                trajstep = np.append(trajstep,[op],axis=0)
                trial_path = np.append(trial_path,[trajstep],axis=0)
                path_length +=1
                
                if path_length > lmax:
                    reject_move = True
                    break

        # Final chance to reject a path (because no crossing of interface i)
        if np.where(trial_path[:,6]>interfaces[i])[0].shape[0] == 0:
            reject_move = True
        
        # If we DON'T reject, then path becomes trial path
        if reject_move == False:
            path = trial_path
            acc[i] += 1
        
        # Here we count the crosses, ignoring all paths before move 50
        if move > n_discard:
            if np.where(path[:,6]>interfaces[i+1])[0].shape[0] > 0:
                n_cross[i] += 1
        allpaths[i].append(path)

endtime = time.monotonic()

print('Total simulation time for {} moves: {:10.3f} s'.format(n_moves,endtime-starttime))

## Plot the paths collected at each interface

In [ ]:
# Plot sampled paths from each interface
skip = 5

for i in range(len(interfaces) - 1):
    plt.figure()
    
    plt.contour(x_vec,y_vec,energy,np.linspace(-3,3,20), cmap = 'jet')

    for j in range(1,len(allpaths[i]),skip):
        plt.plot(allpaths[i][j][:,0],allpaths[i][j][:,1])                                   # allpaths[i] tells you the interface ensemble you're in, 
                                                                                            # [j] tells you a specific path in that ensemble, [:,0/1] 
                                                                                            # takes all the x/y values of the paths in i
    
    # Plot basin boundaries

    if op_type == 1:
        plt.plot(np.linspace(basinA,basinA,10),np.linspace(min(y_vec),max(y_vec),10),color='r', label='Basin A')
        plt.plot(np.linspace(basinB,basinB,10),np.linspace(min(y_vec),max(y_vec),10),color='b', label='Basin B')
    elif op_type == 2:
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinA,basinA,10),color='r', label='Basin A')
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(basinB,basinB,10),color='b', label='Basin B')
    else:
        xplot = np.linspace(min(x_vec),max(x_vec),10)
        yplmax = basinB - xplot
        yplmin = basinA - xplot
        plt.plot(xplot,yplmin,color='r', label='Basin A')
        plt.plot(xplot,yplmax,color='b', label='Basin B')
    
    # Plot interfaces
    if op_type == 1:
        plt.plot(np.linspace(interfaces[i],interfaces[i],10),np.linspace(min(y_vec),max(y_vec),10), color='grey')
    elif op_type == 2:
        plt.plot(np.linspace(min(x_vec),max(x_vec),10),np.linspace(interfaces[i],interfaces[i],10), color='grey')
    else:
        xplot = np.linspace(min(x_vec),max(x_vec),10)
        yplot = interfaces[i]-xplot
        plt.plot(xplot,yplot, color='grey')
        
    cbar = plt.colorbar(cm.ScalarMappable(cmap='jet'))
    cbar.set_ticks([])
    cbar.set_label(label = 'Energy', size=12)
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show
    

## Visualize density of configurations sampled

In [ ]:
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn

skip = 1

x = []
y = [] 

for l in range(len(interfaces)-1):
    for i in range(len(allpaths[l])):
        # Let's only look at transition paths
        if np.where(allpaths[l][i][:,6] <= basinA)[0].shape[0] == 0 or np.where(allpaths[l][i][:,6] >= basinB)[0].shape[0] == 0:
            continue
        else:
            for j in range(len(allpaths[l][i])): 
                x.append(allpaths[l][i][j][0])
                y.append(allpaths[l][i][j][1])
y = np.asarray(y)
x = np.asarray(x)


bins = 20
sort = True
fig , ax = plt.subplots(figsize=(8,6))
data , x_e, y_e = np.histogram2d( x, y, bins = bins, density = True )
z = interpn( ( 0.5*(x_e[1:] + x_e[:-1]) , 0.5*(y_e[1:]+y_e[:-1]) ) , data , np.vstack([x,y]).T , method = "splinef2d", bounds_error = False)

#To be sure to plot all data
z[np.where(np.isnan(z))] = 0.0

# Sort the points by density, so that the densest points are plotted last
if sort :
    idx = z.argsort()
    x, y, z = x[idx], y[idx], z[idx]

ax.contour(x_vec,y_vec,energy,[-3.5,-3,-2.5,-2,-1.5,-1,-0.5,0,0.5,1,1.5,2,2.5,3], colors='silver')
ax.scatter( x, y, c=z, s=2, cmap = 'jet', alpha = 0.002)

norm = Normalize(vmin = np.min(z), vmax = np.max(z))
cbar = fig.colorbar(cm.ScalarMappable(norm = norm, cmap = 'jet'), ax=ax)
cbar.ax.set_ylabel('Density')         

ax.set_xlabel('x',fontsize=15)
ax.set_ylabel('y',fontsize=15)
ax.tick_params(axis='both',labelsize=12)
plt.show()

## Calculate the flux, acceptance probabilities, crossing probabilities and rate constant

- ## Equations: 
    $$ k_{AB} = \Phi_{\lambda_{0}} * P(\lambda_{B}|\lambda_{A}) = \Phi_{\lambda_{0}} * \prod_{i=0}^{N_{interface}}{P(\lambda_{i+1}|\lambda_{i})} $$
    $$ Crossing\ Probability = P(\lambda_{B}|\lambda_{A}) = \prod_{i=0}^{N_{interface}}{P(\lambda_{i+1}|\lambda_{i})} $$
    $$ Flux\ through\ the\ 1st\ interface = \Phi_{\lambda_{0}} = \frac{n_{crosses,\lambda_{0}}}{n_{\ tot paths,\ basin}} $$
    $$ P_{\ acc,\ shoot} = \frac{accepted\ moves}{total\ moves}$$

In [ ]:
# Calculate flux

flux = n_cross_flux/(basinlen*dt)
print("Flux through first interface: {}\n".format(flux))
print("Number of first crossings: {}\n".format(n_cross_flux))

# Calculate Acceptance and Crossing Probability 
cross_probs = []

for i in range(len(interfaces)-1):
    cross_prob = n_cross[i]/(len(allpaths[i]) - n_discard)
    cross_probs.append(cross_prob)
    print("Interface %d acceptance prob: %f" % (i, acc[i]/n_moves))

ratek = flux*np.prod(np.asarray(cross_probs))  # flux from the first crossing * product of crossing probability of all interface
print("Crossing probabilities: {}".format(cross_probs))
print("Rate Constant: {:e}".format(ratek))



## Rate Constant Answer Key

In [ ]:
"""" Rate Constant Answer Key """

# For beta = 2.5

# For PES-1: k_AB = 2.9 x 10^5
# For PES-2: k_AB = ???
# For PES-3: k_AB = 2.6 x 10^5
# For PES-4: k_AB = ???



## Plot Cumulative Crossing Probability from A to B

In [ ]:
# Plot cumulative crossing probability
cumuprob = []
for i in range(len(cross_probs)+1):
    cumuprob.append(np.prod(np.asarray(cross_probs[:i])))       # np.asarray(cross_probs[:i]) takes the cross_prob array up until interface i
                                                                # cumuprob[0] then is the probability to cross lambda 0, cumuprob[1] then is the probability to cross lambda 0 AND 1
probs = plt.figure(2)
plt.plot(interfaces,cumuprob)
plt.yscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('$P(\lambda_i|\lambda_0)$')
plt.show

## Calculate the crossing probability at finer intervals of the order parameter

In [ ]:
# Plot the crossing probability of lambda at every 0.1 interval instead of at each interface
cumuprob = []
for i in range(len(interfaces)-1):
    for opval in np.arange(interfaces[i], interfaces[i+1], 0.05):
        count = 0
        for path in allpaths[i]:
            if len(np.where(path[:,6] > opval)[0]) > 0: # if there's a path whose op value is higher than the interface it's crossing then, we consider that the path cross that interface
                count += 1
        op_prob = count/(len(allpaths[i])) # for each i, this is calculating the number of crossing of that interface / total number of paths in that interface ensemble
        op_cumuprob = np.prod(np.asarray(cross_probs[:i]))*op_prob
        cumuprob.append([opval, op_cumuprob])
        
cumuprob = np.asarray(cumuprob)
        
probs = plt.figure(2)
plt.plot(cumuprob[:,0],cumuprob[:,1])
plt.yscale('log')
plt.xlabel('$\lambda$')
plt.ylabel('$P(\lambda_i|\lambda_0)$')
plt.show